In [2]:

#import dependencies
import pandas as pd
import requests
import time
import json
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func,inspect

In [ ]:
df_BOX_PLOT = pd.read_csv('BoxPlotData.csv') 

In [ ]:
#Pull full dataset from Fantasy API and in json pretty format

query_url = f"https://api.sportsdata.io/v3/nfl/stats/json/FantasyPlayers?key=4339cb9aa27441699f1883848f498899"
response = requests.get(query_url).json()
print(json.dumps(response,indent=1))

In [3]:
#Create Data Frame from API results (includes all fields)
df= pd.read_json("https://api.sportsdata.io/v3/nfl/stats/json/FantasyPlayers?key=4339cb9aa27441699f1883848f498899")

In [4]:
df.head()

,FantasyPlayerKey,PlayerID,Name,Team,Position,AverageDraftPosition,AverageDraftPositionPPR,ByeWeek,LastSeasonFantasyPoints,ProjectedFantasyPoints,AuctionValue,AuctionValuePPR,AverageDraftPositionIDP,AverageDraftPositionRookie,AverageDraftPositionDynasty,AverageDraftPosition2QB
0,18877,18877,Christian McCaffrey,CAR,RB,1.7,1.7,13.0,82.8,310.7,77,77,4.0,NaN,NaN,NaN
1,17959,17959,Derrick Henry,TEN,RB,2.7,3.9,13.0,354.3,301.4,69,60,46.0,NaN,NaN,NaN
2,18872,18872,Dalvin Cook,MIN,RB,3.2,3.2,7.0,331.4,299.9,71,70,19.0,NaN,NaN,NaN
3,18878,18878,Alvin Kamara,NO,RB,5.5,5.2,6.0,332.5,257.6,59,60,3.0,NaN,NaN,NaN
4,19798,19798,Nick Chubb,CLE,RB,7.1,9.5,13.0,216.2,262.1,58,49,20.0,NaN,NaN,NaN


In [5]:
#BY
#Create Data Frame that will be used to create a table of just positions that wil be used for site drop down
position = df["Position"].unique()
position_dropdown_df = pd.DataFrame(position).rename(columns={0:'Position'})
position_dropdown_df
# position_df =pd.DataFrame(position)

,Position
0,RB
1,WR
2,TE
3,QB
4,DEF
5,K


In [6]:
#BY
#Create Data Frame that will be used to create table of all stats for each posiiton (5 total)
#RB

RB = df.drop(columns =["FantasyPlayerKey", "PlayerID","AuctionValue","AuctionValuePPR","AverageDraftPositionIDP","AverageDraftPositionRookie","AverageDraftPositionDynasty","AverageDraftPosition2QB"])
RB_df = RB.loc[RB["Position"]=="RB"]
RB_df
# # RB_df = df["Position"].u
# oc[concatenated_df["3_TECH_ORG_CORRECT"]=="1"]nique()

# tech_mental_health = concatenated_df.loc[concatenated_df["3_TECH_ORG_CORRECT"]=="1"]
# tech_mental_health

# position_df = pd.DataFrame(position).rename(columns={0:'Position'})
# position_df

# "Team","Position","AverageDraftPosition", "AverageDraftPositionPPR", "ByeWeek", "LastSeasonFantasyPoints", "ProjectedFantasyPoints"\
#                 from "ADP_Data"\

,Name,Team,Position,AverageDraftPosition,AverageDraftPositionPPR,ByeWeek,LastSeasonFantasyPoints,ProjectedFantasyPoints
0,Christian McCaffrey,CAR,RB,1.7,1.7,13.0,82.8,310.6
1,Derrick Henry,TEN,RB,2.7,4.7,13.0,354.3,301.4
2,Dalvin Cook,MIN,RB,3.2,3.3,7.0,331.4,299.9
3,Alvin Kamara,NO,RB,5.3,5.0,6.0,332.5,257.6
4,Nick Chubb,CLE,RB,7.3,9.5,13.0,216.2,262.1
...,...,...,...,...,...,...,...,...
859,Spencer Brown,CAR,RB,2178.2,2184.9,13.0,0.0,0.0
860,Deon Jackson,IND,RB,2179.3,2186.1,14.0,0.0,0.0
861,C.J. Marable,CHI,RB,2180.4,2187.2,10.0,0.0,0.0
869,Taquan Mizzell,None,RB,2224.4,2231.2,NaN,17.4,0.0


In [7]:
#Create Highlights table: to appear on Home page; extract players from positions with lowest ADP for the week
highlights = df[["Name", "Team", "Position","AverageDraftPosition","AverageDraftPositionPPR","ByeWeek","LastSeasonFantasyPoints","ProjectedFantasyPoints"]].groupby("Position", as_index=False).apply(lambda df:df.sort_values("AverageDraftPosition", ascending=True)
.head(1)).droplevel(0).sort_values("Position", ascending=True)
highlights

,Name,Team,Position,AverageDraftPosition,AverageDraftPositionPPR,ByeWeek,LastSeasonFantasyPoints,ProjectedFantasyPoints
105,Pittsburgh Steelers,PIT,DEF,117.1,121.9,7.0,169.2,129.7
145,Harrison Butker,KC,K,152.6,160.2,12.0,147.8,159.3
16,Patrick Mahomes,KC,QB,17.3,23.3,12.0,422.3,448.7
0,Christian McCaffrey,CAR,RB,1.7,1.7,13.0,82.8,310.6
11,Travis Kelce,KC,TE,12.5,10.7,12.0,234.4,218.3
9,Tyreek Hill,KC,WR,10.3,12.6,12.0,272.9,241.7


In [8]:
#BY
#Create Data Frame that will be used to create table of all stats for each posiiton (5 total)
#WR 

WR = df.drop(columns =["FantasyPlayerKey", "PlayerID","AuctionValue","AuctionValuePPR","AverageDraftPositionIDP","AverageDraftPositionRookie","AverageDraftPositionDynasty","AverageDraftPosition2QB"])
WR_df = WR.loc[WR["Position"]=="WR"]
WR_df

,Name,Team,Position,AverageDraftPosition,AverageDraftPositionPPR,ByeWeek,LastSeasonFantasyPoints,ProjectedFantasyPoints
9,Tyreek Hill,KC,WR,10.3,12.6,12.0,272.9,241.7
10,Davante Adams,GB,WR,11.8,8.0,13.0,274.6,248.1
15,Stefon Diggs,BUF,WR,16.6,14.4,7.0,227.4,196.4
19,DK Metcalf,SEA,WR,20.8,22.3,9.0,212.4,216.4
21,DeAndre Hopkins,ARI,WR,23.0,18.3,12.0,194.9,203.8
...,...,...,...,...,...,...,...,...
856,Bennie Fowler,None,WR,958.8,965.6,NaN,1.4,0.0
870,Jeff Badet,None,WR,2225.5,2232.3,NaN,0.0,0.0
871,Matt Cole,None,WR,2227.8,2234.6,NaN,0.0,0.0
874,Trenton Irwin,CIN,WR,2274.0,769.3,10.0,0.6,-0.2


In [9]:
#BY
#Create Data Frame that will be used to create table of all stats for each posiiton (5 total)
#TE

TE = df.drop(columns =["FantasyPlayerKey", "PlayerID","AuctionValue","AuctionValuePPR","AverageDraftPositionIDP","AverageDraftPositionRookie","AverageDraftPositionDynasty","AverageDraftPosition2QB"])
TE_df = TE.loc[TE["Position"]=="TE"]
TE_df

,Name,Team,Position,AverageDraftPosition,AverageDraftPositionPPR,ByeWeek,LastSeasonFantasyPoints,ProjectedFantasyPoints
11,Travis Kelce,KC,TE,12.5,10.7,12.0,234.4,218.3
27,Darren Waller,LV,TE,30.2,25.3,8.0,193.6,173.3
30,George Kittle,SF,TE,34.0,31.0,6.0,87.0,167.0
44,Kyle Pitts,ATL,TE,49.4,44.4,6.0,0.0,137.4
55,Mark Andrews,BAL,TE,60.7,59.7,8.0,126.4,148.1
...,...,...,...,...,...,...,...,...
862,Quintin Morris,BUF,TE,2181.6,2188.3,7.0,0.0,0.0
863,Pro Wells,CIN,TE,2182.7,2189.4,10.0,0.0,0.0
864,Nick Eubanks,DAL,TE,2183.8,2190.6,7.0,0.0,0.0
865,Matt Bushman,LV,TE,2184.9,2191.7,8.0,0.0,0.0


In [10]:

#BY
#Create Data Frame that will be used to create table of all stats for each posiiton (5 total)
#QB

QB = df.drop(columns =["FantasyPlayerKey", "PlayerID","AuctionValue","AuctionValuePPR","AverageDraftPositionIDP","AverageDraftPositionRookie","AverageDraftPositionDynasty","AverageDraftPosition2QB"])
QB_df = QB.loc[QB["Position"]=="QB"]
QB_df

,Name,Team,Position,AverageDraftPosition,AverageDraftPositionPPR,ByeWeek,LastSeasonFantasyPoints,ProjectedFantasyPoints
16,Patrick Mahomes,KC,QB,17.3,23.3,12.0,422.3,448.7
28,Josh Allen,BUF,QB,31.7,40.0,7.0,445.6,432.2
37,Kyler Murray,ARI,QB,41.3,48.5,12.0,427.2,409.4
45,Dak Prescott,DAL,QB,49.9,64.2,7.0,151.9,398.4
47,Lamar Jackson,BAL,QB,51.7,54.9,8.0,375.4,399.7
...,...,...,...,...,...,...,...,...
837,Mike White,NYJ,QB,928.3,935.1,6.0,0.0,0.0
838,Jake Browning,MIN,QB,929.5,936.2,7.0,0.0,0.0
852,Blake Bortles,None,QB,953.2,959.9,NaN,0.0,0.0
867,Brian Hoyer,NE,QB,2187.2,2194.0,14.0,2.5,0.0


In [11]:
DEF = df.drop(columns =["FantasyPlayerKey", "PlayerID","AuctionValue","AuctionValuePPR","AverageDraftPositionIDP","AverageDraftPositionRookie","AverageDraftPositionDynasty","AverageDraftPosition2QB"])
DEF_df = DEF.loc[DEF["Position"]=="DEF"]
DEF_df

,Name,Team,Position,AverageDraftPosition,AverageDraftPositionPPR,ByeWeek,LastSeasonFantasyPoints,ProjectedFantasyPoints
105,Pittsburgh Steelers,PIT,DEF,117.1,121.9,7.0,169.2,129.7
112,Los Angeles Rams,LAR,DEF,122.2,128.7,11.0,168.1,115.1
120,Baltimore Ravens,BAL,DEF,131.6,148.0,8.0,161.3,121.8
122,Washington Football Team,WAS,DEF,133.0,137.5,9.0,143.3,100.4
126,Tampa Bay Buccaneers,TB,DEF,138.7,132.8,9.0,132.0,107.2
140,New York Giants,NYG,DEF,150.6,192.2,10.0,116.2,91.4
142,Indianapolis Colts,IND,DEF,151.8,156.5,14.0,170.3,116.2
147,San Francisco 49ers,SF,DEF,153.1,160.0,6.0,99.3,118.4
149,Seattle Seahawks,SEA,DEF,156.0,171.1,9.0,116.2,104.9
151,Dallas Cowboys,DAL,DEF,157.4,177.1,7.0,88.0,97.0


In [12]:
K = df.drop(columns =["FantasyPlayerKey", "PlayerID","AuctionValue","AuctionValuePPR","AverageDraftPositionIDP","AverageDraftPositionRookie","AverageDraftPositionDynasty","AverageDraftPosition2QB"])
K_df = DEF.loc[K["Position"]=="K"]
K_df

,Name,Team,Position,AverageDraftPosition,AverageDraftPositionPPR,ByeWeek,LastSeasonFantasyPoints,ProjectedFantasyPoints
145,Harrison Butker,KC,K,152.6,160.2,12.0,147.8,159.3
152,Justin Tucker,BAL,K,157.5,155.6,8.0,153.4,159.3
153,Younghoe Koo,ATL,K,158.9,157.4,6.0,180.5,149.2
156,Rodrigo Blankenship,IND,K,162.0,172.4,14.0,159.0,146.0
158,Greg Zuerlein,DAL,K,162.3,164.2,7.0,159.0,151.7
162,Jason Sanders,MIA,K,165.6,166.3,14.0,180.5,150.2
163,Ryan Succop,TB,K,165.6,170.7,9.0,155.7,154.2
167,Matt Prater,ARI,K,167.7,178.2,12.0,127.5,142.4
168,Tyler Bass,BUF,K,167.8,171.3,7.0,168.1,155.4
180,Wil Lutz,NO,K,176.4,174.4,6.0,144.4,147.5


In [5]:
#Load DF into Database NFL_Fantasy_Data
#Create connection to Postgres
connection_string = "kkcrgtjwdqbzec:1ac1f9f8b00464b596bca48ac558948a1f54554ef23b3b41a8e65ef24e267bdc@ec2-34-204-128-77.compute-1.amazonaws.com:5432/dppc2qup827s8"
engine = create_engine(f'postgresql://{connection_string}')

In [17]:
df.to_sql(name='ADP_Data', con=engine, if_exists='append', index=False)
# engine.execute('ALTER TABLE users ADD PRIMARY KEY (`FantasyPlayerKey`);')

In [18]:
# Create ADP table and load into DB
df_BOX_PLOT.to_sql(name='BoxPlot', con=engine, if_exists='append', index=False)
engine.execute('ALTER TABLE users ADD PRIMARY KEY (`NAME`);')

NameError: name 'df_BOX_PLOT' is not defined

In [20]:
#BY
#Create Dropdown table and load into DB
position_dropdown_df.to_sql(name='Position_dropdown', con=engine, if_exists='append', index=False) 
# engine.execute('ALTER TABLE users ADD PRIMARY KEY (`Position`);')

In [21]:
#BY
#Create  table and load into DB
RB_df.to_sql(name='RB_Data', con=engine, if_exists='append', index=False)
# engine.execute('ALTER TABLE users ADD PRIMARY KEY (`Name`);')

In [22]:
#BY
#Create  table and load into DB
WR_df.to_sql(name='WR_Data', con=engine, if_exists='append', index=False)
# engine.execute('ALTER TABLE users ADD PRIMARY KEY (`Name`);')

In [23]:
#BY
#Create  table and load into DB
TE_df.to_sql(name='TE_Data', con=engine, if_exists='append', index=False)
# engine.execute('ALTER TABLE users ADD PRIMARY KEY (`Name`);')

In [24]:
#BY
#Create  table and load into DB
QB_df.to_sql(name='QB_Data', con=engine, if_exists='append', index=False)
# engine.execute('ALTER TABLE users ADD PRIMARY KEY (`Name`);')

In [25]:
#BY
#Create DEF table and load into DB
DEF_df.to_sql(name='DEF_Data', con=engine, if_exists='append', index=False)
# engine.execute('ALTER TABLE users ADD PRIMARY KEY (`Name`);')

In [26]:
#BY
#Create K table and load into DB
K_df.to_sql(name='K_Data', con=engine, if_exists='append', index=False)
# engine.execute('ALTER TABLE users ADD PRIMARY KEY (`Name`);')

In [27]:
#Create highlights table and load into DB
highlights.to_sql(name='Highlights_Data', con=engine, if_exists='append', index=False)
# engine.execute('ALTER TABLE users ADD PRIMARY KEY (`Name`);')

In [6]:
#confirm data succesfully loaded in
pd.read_sql_query('select * from "ADP_Data"', con=engine)

,FantasyPlayerKey,PlayerID,Name,Team,Position,AverageDraftPosition,AverageDraftPositionPPR,ByeWeek,LastSeasonFantasyPoints,ProjectedFantasyPoints,AuctionValue,AuctionValuePPR,AverageDraftPositionIDP,AverageDraftPositionRookie,AverageDraftPositionDynasty,AverageDraftPosition2QB
0,18877,18877,Christian McCaffrey,CAR,RB,1.7,1.7,13.0,82.8,310.6,77,77,4.0,None,None,None
1,17959,17959,Derrick Henry,TEN,RB,2.7,4.7,13.0,354.3,301.4,71,60,46.0,None,None,None
2,18872,18872,Dalvin Cook,MIN,RB,3.2,3.3,7.0,331.4,299.9,71,70,19.0,None,None,None
3,18878,18878,Alvin Kamara,NO,RB,5.3,5.0,6.0,332.5,257.6,59,60,3.0,None,None,None
4,19798,19798,Nick Chubb,CLE,RB,7.3,9.5,13.0,216.2,262.1,58,50,20.0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
871,22340,22340,Matt Cole,None,WR,2227.8,2234.6,NaN,0.0,0.0,0,0,NaN,None,None,None
872,18132,18132,Darius Jackson,None,RB,2253.7,2260.5,NaN,2.0,0.0,0,0,NaN,None,None,None
873,16850,16850,Sean Mannion,SEA,QB,2254.9,2261.6,9.0,0.0,0.0,0,0,NaN,None,None,None
874,21263,21263,Trenton Irwin,CIN,WR,2274.0,769.3,10.0,0.6,-0.2,0,0,NaN,None,None,None


In [29]:
# Query to find the unique positions of each player; to be used for Landing Page Dropdown
pd.read_sql_query('Select distinct "Position"from "ADP_Data";', con=engine)

,Position
0,DEF
1,RB
2,QB
3,K
4,WR
5,TE


In [7]:
#Note: We are creating views and displaying stats based on the Draft Pick method of selection instead of auction method. We used all position values instead of looking at individual position values exclusively (IDP)
# Query to find the stats per each position once drop down in Landing Page is selected. 
# Want this sorted by ADP Ranking; replicate this query for each position  

#DEF

wr_query = """
Select "Name", "Team","Position","AverageDraftPosition", "AverageDraftPositionPPR", "ByeWeek", "LastSeasonFantasyPoints", "ProjectedFantasyPoints"\
                from "ADP_Data"\
                where "Position"= 'WR' order by "AverageDraftPosition" asc;
"""

pd.read_sql_query(wr_query, con=engine)

,Name,Team,Position,AverageDraftPosition,AverageDraftPositionPPR,ByeWeek,LastSeasonFantasyPoints,ProjectedFantasyPoints
0,Tyreek Hill,KC,WR,10.3,12.6,12.0,272.9,241.7
1,Davante Adams,GB,WR,11.8,8.0,13.0,274.6,248.1
2,Stefon Diggs,BUF,WR,16.6,14.4,7.0,227.4,196.4
3,DK Metcalf,SEA,WR,20.8,22.3,9.0,212.4,216.4
4,DeAndre Hopkins,ARI,WR,23.0,18.3,12.0,194.9,203.8
...,...,...,...,...,...,...,...,...
306,Bennie Fowler,None,WR,958.8,965.6,NaN,1.4,0.0
307,Jeff Badet,None,WR,2225.5,2232.3,NaN,0.0,0.0
308,Matt Cole,None,WR,2227.8,2234.6,NaN,0.0,0.0
309,Trenton Irwin,CIN,WR,2274.0,769.3,10.0,0.6,-0.2


In [31]:
# #Create connection to Postgres
# connection_string = "postgres:postgres@localhost:5432/NFL_Fantasy_Data"
# engine = create_engine(f'postgresql://{connection_string}')

In [8]:

Base = automap_base()

In [9]:
Base.prepare(engine, reflect= True)

In [10]:
Base.classes.keys()

[]

In [11]:
Base.classes

In [53]:
for key in Base.classes.keys():
    print(key)

In [12]:
# Use this instead
Base.metadata.tables.keys()

dict_keys(['ADP_Data', 'Position_dropdown', 'RB_Data', 'WR_Data', 'TE_Data', 'QB_Data', 'DEF_Data', 'K_Data', 'Highlights_Data'])